In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from env import host, user, password
import acquire
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

The objective of this notebook is to inspect the dataset and see if:
- 1.) There is any duplicate values, not in every field but where it would matter.
- 2.) Look for duplicate fields and drop those who are repeated.
- 3.) Inspect fields which could be split into a numerical category.
- 4.) Create dummy variables for those fields we wish to split into a numeric category.
- 5.) Add the dummy columns to the original df!
- 6.) Generate functions within this notebook which will be implemented with in the prepare.py
- 7.) Give a few charts to display data. Pre-exploratory!!!

In [2]:
telco_churn = acquire.get_telco_churn_data()

### Summarize our data:
- head(), describe(), info(), isnull(), value_counts(), shape, ...
- plt.hist(), plt.boxplot()
- document takeaways (nulls, datatypes to change, outliers, ideas for features, etc.)


Import the data and inspect it for nulls, duplicate values and columns, review the data types.

In [3]:
telco_churn.head(2)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,internet_service_type_id,internet_service_type,payment_type_id,payment_type
0,0057-QBUQH,Female,0,No,Yes,43,Yes,Yes,3,No internet service,...,1,25.1,1070.15,No,3,Two year,3,None,1,Electronic check
1,0106-UGRDO,Female,0,Yes,No,69,Yes,Yes,2,Yes,...,1,116.0,8182.85,No,3,Two year,3,None,1,Electronic check


In [4]:
# Displaying all of the datasets columns.
telco_churn.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service_type_id',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract_type_id',
       'paperless_billing', 'payment_type_id', 'monthly_charges',
       'total_charges', 'churn', 'contract_type_id', 'contract_type',
       'internet_service_type_id', 'internet_service_type', 'payment_type_id',
       'payment_type'],
      dtype='object')

In [5]:
# start looking into the dataset.
telco_churn.info()
# appears there is no nulls within the data set. Need to check and make sure no values are nan as well!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               7043 non-null   object 
 1   gender                    7043 non-null   object 
 2   senior_citizen            7043 non-null   int64  
 3   partner                   7043 non-null   object 
 4   dependents                7043 non-null   object 
 5   tenure                    7043 non-null   int64  
 6   phone_service             7043 non-null   object 
 7   multiple_lines            7043 non-null   object 
 8   internet_service_type_id  7043 non-null   int64  
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

In [6]:
telco_drop = telco_churn.drop_duplicates(inplace = True)

In [7]:
telco_churn.set_index('customer_id', drop = True, inplace = True)

In [8]:
telco_churn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0057-QBUQH to 9974-JFBHQ
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    7043 non-null   object 
 1   senior_citizen            7043 non-null   int64  
 2   partner                   7043 non-null   object 
 3   dependents                7043 non-null   object 
 4   tenure                    7043 non-null   int64  
 5   phone_service             7043 non-null   object 
 6   multiple_lines            7043 non-null   object 
 7   internet_service_type_id  7043 non-null   int64  
 8   online_security           7043 non-null   object 
 9   online_backup             7043 non-null   object 
 10  device_protection         7043 non-null   object 
 11  tech_support              7043 non-null   object 
 12  streaming_tv              7043 non-null   object 
 13  streaming_movies          7043 non-null   object 
 14

In [9]:
#are there any empty records??
telco_churn.replace(' ', np.nan, inplace = True)

In [10]:
#double checking the dataset for nulls
# appears total charges has 11 nan values.
nan_check = telco_churn.isna().sum()

In [11]:
#dropped the nan values within the total charges column
telco_churn.dropna(inplace =True)
telco_churn.count()

gender                      7032
senior_citizen              7032
partner                     7032
dependents                  7032
tenure                      7032
phone_service               7032
multiple_lines              7032
internet_service_type_id    7032
online_security             7032
online_backup               7032
device_protection           7032
tech_support                7032
streaming_tv                7032
streaming_movies            7032
contract_type_id            7032
paperless_billing           7032
payment_type_id             7032
monthly_charges             7032
total_charges               7032
churn                       7032
contract_type_id            7032
contract_type               7032
internet_service_type_id    7032
internet_service_type       7032
payment_type_id             7032
payment_type                7032
dtype: int64

In [12]:
# statistical details of the dataset.
telco_churn.describe().T

,count,mean,std,min,25%,50%,75%,max
senior_citizen,7032.0,0.162400,0.368844,0.00,0.0000,0.00,0.0000,1.00
tenure,7032.0,32.421786,24.545260,1.00,9.0000,29.00,55.0000,72.00
internet_service_type_id,7032.0,1.872582,0.737271,1.00,1.0000,2.00,2.0000,3.00
contract_type_id,7032.0,1.688567,0.832934,1.00,1.0000,1.00,2.0000,3.00
payment_type_id,7032.0,2.315557,1.149523,1.00,1.0000,2.00,3.0000,4.00
monthly_charges,7032.0,64.798208,30.085974,18.25,35.5875,70.35,89.8625,118.75
contract_type_id,7032.0,1.688567,0.832934,1.00,1.0000,1.00,2.0000,3.00
internet_service_type_id,7032.0,1.688567,0.832934,1.00,1.0000,1.00,2.0000,3.00
payment_type_id,7032.0,2.315557,1.149523,1.00,1.0000,2.00,3.0000,4.00


## Begin cleaning up the table and data

In [13]:
# dropped duplicate columns.
telco_churn = telco_churn.loc[:, ~telco_churn.columns.duplicated()]

In [14]:
telco_churn = telco_churn.astype({'total_charges': 'float64'})

In [15]:
telco_churn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0057-QBUQH to 9974-JFBHQ
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    7032 non-null   object 
 1   senior_citizen            7032 non-null   int64  
 2   partner                   7032 non-null   object 
 3   dependents                7032 non-null   object 
 4   tenure                    7032 non-null   int64  
 5   phone_service             7032 non-null   object 
 6   multiple_lines            7032 non-null   object 
 7   internet_service_type_id  7032 non-null   int64  
 8   online_security           7032 non-null   object 
 9   online_backup             7032 non-null   object 
 10  device_protection         7032 non-null   object 
 11  tech_support              7032 non-null   object 
 12  streaming_tv              7032 non-null   object 
 13  streaming_movies          7032 non-null   object 
 14

In [16]:
telco_churn.head().T

customer_id,0057-QBUQH,0106-UGRDO,0136-IFMYD,0164-APGRB,0253-ZTEOB
gender,Female,Female,Male,Female,Female
senior_citizen,0,0,1,0,0
partner,No,Yes,Yes,No,Yes
dependents,Yes,No,No,No,Yes
tenure,43,69,69,72,48
phone_service,Yes,Yes,Yes,Yes,Yes
multiple_lines,Yes,Yes,Yes,Yes,Yes
internet_service_type_id,3,2,2,2,1
online_security,No internet service,Yes,No,Yes,No
online_backup,No internet service,Yes,Yes,Yes,Yes


In [16]:
telco_churn.multiple_lines.value_counts()

No                  3385
Yes                 2967
No phone service     680
Name: multiple_lines, dtype: int64

In [18]:
telco_churn.gender = telco_churn.gender.replace({'Female': 1, 'Male':0})
telco_churn.partner = telco_churn.partner.replace({'Yes': 1, 'No':0})
telco_churn.dependents = telco_churn.dependents.replace({'Yes': 1, 'No':0})

In [19]:
telco_churn.phone_service= telco_churn.phone_service.replace({'Yes': 1, 'No':0})
telco_churn.multiple_lines = telco_churn.multiple_lines.replace({'Yes': 1, 'No':0, 'No phone service': 2})

In [20]:
telco_churn.online_security = telco_churn.online_security.replace({'Yes': 1, 'No':0, 'No internet service': 2})

In [21]:
telco_churn.online_backup = telco_churn.online_backup.replace({'Yes': 1, 'No':0, 'No internet service': 2})

In [22]:
telco_churn.device_protection = telco_churn.device_protection.replace({'Yes': 1, 'No':0, 'No internet service': 2})

In [23]:
telco_churn.tech_support = telco_churn.tech_support.replace({'Yes': 1, 'No':0, 'No internet service': 2})

In [24]:
telco_churn.streaming_tv = telco_churn.streaming_tv.replace({'Yes': 1, 'No':0, 'No internet service': 2})

In [25]:
telco_churn.streaming_movies = telco_churn.streaming_movies.replace({'Yes': 1, 'No':0, 'No internet service': 2})

In [26]:
telco_churn.paperless_billing = telco_churn.paperless_billing.replace({'Yes': 1, 'No':0})

In [27]:
telco_churn.contract_type = telco_churn.contract_type.replace({'Month-to-month': 0, 'One Year':1, 'Two year': 2})

In [28]:
telco_churn.internet_service_type = telco_churn.internet_service_type.replace({'None': 0, 'DSL':1, 'Fiber optic': 2})

In [39]:
telco_churn.payment_type = telco_churn.payment_type.replace({'Bank transfer (automatic)': 0, 'Credit card (automatic)':1, 'Electronic check': 2, 'Mailed check': 3})

In [30]:
telco_churn.churn = telco_churn.churn.replace({'Yes': 1, 'No':0})

In [38]:
telco_churn.head(4)

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
customer_id,,,,,,,,,,,,,,,,,,,,,
0057-QBUQH,1,0,0,1,43,1,1,3,2,2,...,2,3,1,1,25.10,1070.15,0,2,0,2
0106-UGRDO,1,0,1,0,69,1,1,2,1,1,...,1,3,1,1,116.00,8182.85,0,2,0,2
0136-IFMYD,0,1,1,0,69,1,1,2,0,1,...,1,3,1,1,109.95,7634.25,0,2,0,2
0164-APGRB,1,0,0,0,72,1,1,2,1,1,...,1,3,1,1,114.90,8496.70,0,2,0,2


## Creating simple charts to evaluate the data and see if there is any relationships.

In [32]:
#Numeric features
numeric = ['monthly_charges', 'total_charges']
num_cols = telco_churn[numeric]

In [33]:
#Categorical features
object_cols = telco_churn.drop(columns = ['monthly_charges', 'total_charges'])


In [40]:
# Look at the value count for each categorical feature.
for col in object_cols:
    print(col)
    print(telco_churn[col].value_counts())
    print('------------------------------------------------')


gender
0    3549
1    3483
Name: gender, dtype: int64
------------------------------------------------
senior_citizen
0    5890
1    1142
Name: senior_citizen, dtype: int64
------------------------------------------------
partner
0    3639
1    3393
Name: partner, dtype: int64
------------------------------------------------
dependents
0    4933
1    2099
Name: dependents, dtype: int64
------------------------------------------------
tenure
1     613
72    362
2     238
3     200
4     176
     ... 
38     59
28     57
39     56
44     51
36     50
Name: tenure, Length: 72, dtype: int64
------------------------------------------------
phone_service
1    6352
0     680
Name: phone_service, dtype: int64
------------------------------------------------
multiple_lines
0    3385
1    2967
2     680
Name: multiple_lines, dtype: int64
------------------------------------------------
internet_service_type_id
2    3096
1    2416
3    1520
Name: internet_service_type_id, dtype: int64
-----------

In [35]:
#created list of categorical columns which excluded customer id.

In [36]:
# want to change columns with yes and no to numeric 0 and 1

In [37]:
# using above variables to see value counts for each column.